# Configurations

In [ ]:
# These are configuration variable used by the notebook

NUBANK_CATEGORY_CONVERSION = {
    'supermercado': 'mercado',
    'eletrônicos':  'compras',
    'vestuário':    'compras',
    'ajuste':       'outros',
}


SPLITWISE_CATEGORY_CONVERSION = {
    'aluguel':                    'casa',
    'tv/telefone/internet':       'casa',
    'eletricidade':               'casa',
    'manutenção':                 'casa',
    'casa - outros':              'casa',
    'serviços':                   'casa',
    'móveis':                     'casa',
    'jantar fora':                'restaurante',
    'comidas e bebidas - outros': 'restaurante',
    'produtos de limpeza':        'mercado',
    'táxi':                       'transporte',
    'filmes':                     'lazer',
    'música':                     'lazer',
    'bicicleta':                  'lazer',
    'jogos':                      'lazer',
    'entretenimento - outros':    'lazer',
    'esportes':                   'lazer',
    'despesas médicas':           'saúde',
    'eletrônicos':                'compras',
    'hotel':                      'viagem',
    'ônibus/trem':                'viagem',
    'avião':                      'viagem',
    'combustível':                'viagem',
    'carro':                      'viagem',
    'seguro':                     'outros',
    'vida - outros':              'outros',
    'geral':                      'outros',
    'presentes':                  'outros',
}

EXPENSES_DISTRIBUTION = {
    'casa': 0.24,
    'restaurante': 0.09,
    'mercado': 0.09,
    'lazer': 0.02,
    'transporte': 0.02,
    'compras': 0.01,
    'saúde': 0.01
}

# TODO Add jacas keter splitwise group to the expenses
LOAD_CONFIGS = {
    'data_path': "./data", # Directory that contains the expenses data
    'nubank_file_pattern': "nubank*.csv",
    'splitwise_file_pattern': "20*mozi-e-eu*.csv",
    'manual_file_pattern': "Extrato outras contas*Despesas*.csv",
    'incomes_file_pattern': "Extrato outras contas*Rendimentos*.csv",
    'person_who_pays': "Lucas Alencar", # Person name that the expenses will be extracted from Splitwise csv
    'splitwise_category_table': SPLITWISE_CATEGORY_CONVERSION,
    'nubank_category_table': NUBANK_CATEGORY_CONVERSION,
}

from datetime import date, timedelta
from date_helpers import previous_month

# Date used to make month analysis through the notebook
BASE_DATE = previous_month(date.today())

# Year you were born
YEAR_OF_BIRTH = 1991

## Data load and preprocess
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

from IPython.display import display
import record_summary as rs

from load import incomes, expenses

incomes = incomes.load(**LOAD_CONFIGS)
expenses = expenses.load(**LOAD_CONFIGS)

# Monthly Expenses

In [ ]:
from expenses import monthly as monthly_expenses

MONTHLY_BALANCE_GOAL = 0.4
print(" >>> Monthly balance goal:", MONTHLY_BALANCE_GOAL * 100, "%")

monthly_exp = monthly_expenses.month_by_month_summary(expenses, incomes)
display(monthly_expenses.style_summary(monthly_exp, MONTHLY_BALANCE_GOAL))

avg_monthly_exp = monthly_expenses.avg_month_summary(expenses, incomes)
display(monthly_expenses.style_summary(avg_monthly_exp, MONTHLY_BALANCE_GOAL))

monthly_expenses.plot(monthly_exp)

# Expenses by category

In [ ]:
from expenses import category

# BASE_DATE = date(2019, 2, 1)
print(">>> Month of analysis: ", BASE_DATE.month, BASE_DATE.year)

expenses_for_month = rs.records_for_month(expenses, BASE_DATE)
incomes_for_month = rs.records_for_month(incomes, BASE_DATE)

expenses_for_category_by_total = category.distribution(expenses_for_month, incomes_for_month, EXPENSES_DISTRIBUTION)
display(category.style_distribution(expenses_for_category_by_total))

category.display_expenses(expenses_for_month, 'outros')

In [ ]:
from expenses import category

# Plots over time for each category grouping them by easy of read
expenses_over_time_data = category.add_food_expenses(expenses, 
                                                     category.over_time(expenses, incomes, 'amount #'))

from plotting import over_time
over_time.plot(expenses_over_time_data[['casa', 'viagem', 'educação']])
over_time.plot(expenses_over_time_data[['restaurante', 'mercado', 'alimentação']])
over_time.plot(expenses_over_time_data[['transporte', 'lazer', 'serviços']])

# Return progress

In [ ]:
from investments import monthly as monthly_investments
from load import central_bank

# BASE_DATE = date(2019, 3, 1)
print(">>> Month of analysis:", BASE_DATE.month, BASE_DATE.year)

current_ipca = central_bank.ipca_for_month(BASE_DATE)
current_ipca = current_ipca * 100 if current_ipca else None
print("> Current IPCA:", current_ipca, "%")

RETURN_FOR_MONTH_GOAL = 0.005
RETURN_WITH_INFLATION_GOAL = 0.0045

invest = incomes[incomes.category.isin(['aplicação', 'valor aplicado', 'desconto'])]

# Table with returns for the current month
summary_invest = monthly_investments.summary(invest, BASE_DATE).sort_values('Return for month (%)', ascending=False)

display(monthly_investments.style_summary(summary_invest, RETURN_FOR_MONTH_GOAL, RETURN_WITH_INFLATION_GOAL))

# Return over time

In [ ]:
# Get all investments titles
# import pandas as pd
# display(pd.DataFrame(invest['title'].unique(), columns=['title']))

# Some visualizations to use and see the return history for the groups
general = ['Nuconta Renda Fixa 100% CDI',
           'Tesouro Selic 2021',
           'CDB Banco Maxima',
           'Magnetis Diversificação Renda Fixa',
           'Magnetis Diversificação Multimercados',
           'Magnetis Diversificação Ações']

# Other investments in renda fixa are better than Tesouro Selic? They are still worth it?
renda_fixa = ['Nuconta Renda Fixa 100% CDI',
              'Tesouro Selic 2021',
              'Tesouro Selic 2023',
              'Magnetis Diversificação Renda Fixa',
              'LCI Banco Original',
              'CDB Banco Maxima',
              'Brasil Plural Yield Fundo de Investimento Renda Fixa Referen']

# Is Magnetis investing in good stocks? Is the cumulative return increasing?
renda_variavel = ['Magnetis Diversificação Ações',
                  'BRAX11',
                  'Magnetis Diversificação Multimercados']

# Does bitcoin is still a good experiment?
bitcoins = ['Bitcoin', 'Cripto Trade']

# There are better opportunities of investments over these Tesouros?
tesouros = ['Tesouro IPCA+ 2024',
            'Tesouro Selic 2021',
            'Tesouro Selic 2023']

a_year_ago = BASE_DATE - timedelta(days=365)
RETURN_STARTING_MONTH = a_year_ago.strftime('%Y-%m')
RETURN_ENDING_MONTH = BASE_DATE.strftime('%Y-%m')

print('from:', RETURN_STARTING_MONTH, 'to:', RETURN_ENDING_MONTH)

PLOT_TITLE_GROUP = general

import plotting
from investments import over_time

plotting.over_time.plot(over_time.return_percentage_over_time(invest)\
                        .loc[RETURN_STARTING_MONTH:RETURN_ENDING_MONTH, PLOT_TITLE_GROUP], 
               title='Return over time (%)')

plotting.over_time.plot(over_time.cumulative_return_over_time(invest)\
                        .loc[RETURN_STARTING_MONTH:RETURN_ENDING_MONTH, PLOT_TITLE_GROUP], 
               title='Cumulative return over time')

# Investment progress

In [ ]:
from investments import assets

ASSETS_SUMMARY_START_DATE = date(2018, 2, 1)
ASSETS_SUMMARY_END_DATE = BASE_DATE

assets_summary = assets.summary(invest, ASSETS_SUMMARY_START_DATE, ASSETS_SUMMARY_END_DATE)
display(assets.style_summary(assets_summary))

from investments import types
types.plot(invest, BASE_DATE)

# Plot line to see assets progress
from plotting import over_time
over_time.plot(assets_summary['Return'].to_frame(), title='Return')
over_time.plot(assets_summary['Applications'].to_frame(), title='Applications')

# Investments liquidations

In [ ]:
from investments import post_liquidation
## TODO Add ROI to this table
post_liquidation.final_return(post_liquidation.liquidations_incomes(incomes))

# Assets goals

### (PMS) Patrimônio Mínimo de Sobrevivência

Reserva de emergência

```6 * Custo mensal```

### (PMR) Patrimônio Mínimo Recomendado para segurança 

Possibilita decisões mais arriscada sobre carreira ou mudanças.

```12 * Custo mensal```

### (PI) Patrimônio Ideal para idade e consumo 

Dado os meus gastos mensais atuais, se eu continuar trabalhando até a aposentadoria (65 anos), eu conseguiria viver apenas gastando esse dinheiro.

```0.1 * PMR * Idade```

### (PNIF) Patrimônio Necessário para a Independência Financeira

Dado os meus rendimentos com investimentos, eu conseguiria manter o mesmo padrão apenas com o dinheiro investido

```PMR / Rendimento anual %```

In [ ]:
from investments import assets

ASSETS_GOALS_START_DATE = BASE_DATE - timedelta(days=365)
ASSETS_GOALS_END_DATE = BASE_DATE

print("Projected return for a year: {:,.4f}%"\
      .format(assets.projected_return_for_year(invest, ASSETS_GOALS_START_DATE, ASSETS_GOALS_END_DATE) * 100))

expenses_by_month = rs.total_amount_by(rs.groupby_month(expenses), expenses)

display(assets.style_goals(assets.goals(expenses_by_month,
                                        invest, 
                                        ASSETS_GOALS_START_DATE,
                                        ASSETS_GOALS_END_DATE, 
                                        YEAR_OF_BIRTH)))

from plotting import over_time
over_time.plot(assets_summary.loc[:ASSETS_GOALS_END_DATE.strftime('%Y-%m'), 'Total'].to_frame(), 
               title='Assets')

# How much my time is worth?

In [ ]:
from incomes import salary

HOURS_WORKED_BY_DAY = 8.8 # According to CLT

# BASE_DATE = date(2019,7,1)

earned_for_month = salary.earned_for_month(incomes, BASE_DATE)
hours_worked_by_month = salary.hours_worked_for_month(HOURS_WORKED_BY_DAY, BASE_DATE)
money_for_time = earned_for_month / hours_worked_by_month

msg = """In {}, you have earned {}.
Given you have worked for {:,.2f} hours in this month.
Your time is worth {:,.2f} R$/hour.
Not considering holidays or vacations."""

print(msg.format(BASE_DATE.strftime('%Y-%m'), earned_for_month, hours_worked_by_month, money_for_time))